# Demo 1. Create your own MPS!

In [ ]:
import numpy as np
from typing import List, TypeVar

class MPS:
    """
    Matrix Product State class.
    Each tensor has shape (left_bond, physical_dim, right_bond).
    """
    def __init__(self, tensor_list: List[np.ndarray]):
        if not tensor_list:
            raise ValueError("tensor_list cannot be empty")

        # Validate 3D tensors
        for i, tensor in enumerate(tensor_list):
            if tensor.ndim != 3:
                raise ValueError(f"Tensor {i} must be 3D, got {tensor.ndim}D")

        self.tensor_list = tensor_list

    def __len__(self) -> int:
        return len(self.tensor_list)

    def phys_dim(self) -> int:
        return self.tensor_list[0].shape[1]

    def max_bond_dim(self) -> int:
        all_bonds = []
        for tensor in self.tensor_list:
            all_bonds.extend([tensor.shape[0], tensor.shape[2]])
        return max(all_bonds)

    def bond_dims(self) -> List[int]:
        """Return list of all bond dimensions."""
        dims = [self.tensor_list[0].shape[0]]  # First left bond
        for tensor in self.tensor_list:
            dims.append(tensor.shape[2])  # Right bonds
        return dims

    def copy(self) -> 'MPS':
        return MPS([tensor.copy() for tensor in self.tensor_list])

    def __repr__(self) -> str:
        return (f"MPS(sites={len(self)}, "
                f"phys_dim={self.phys_dim()}, "
                f"max_bond={self.max_bond_dim()}, "
                f"dtype={self.tensor_list[0].dtype})")
    def __getitem__(self, index: int) -> np.ndarray:
        """Access tensor at given site."""
        return self.tensor_list[index]

    def __setitem__(self, index: int, tensor: np.ndarray):
        """Set tensor at given site."""
        if tensor.ndim != 3:
            raise ValueError("Tensor must be 3D")
        self.tensor_list[index] = tensor

# Test the simplified version
tensors = [np.random.randn(1, 2, 4),
            np.random.randn(4, 2, 4),
            np.random.randn(4, 2, 1)]

mps = MPS(tensors)
print(f"Real MPS: {mps}")

# Works with any dtype
complex_tensors = [t.astype(np.complex128) for t in tensors]
complex_mps = MPS(complex_tensors)
print(f"Complex MPS: {complex_mps}")

Real MPS: MPS(sites=3, phys_dim=2, max_bond=4, dtype=float64)
Complex MPS: MPS(sites=3, phys_dim=2, max_bond=4, dtype=complex128)


In [ ]:
# Helper functions for creating common MPS states
def create_random_mps(length: int, phys_dim: int, bond_dim: int,
                     dtype: np.dtype = np.float64) -> MPS:
    """
    Create a random MPS with specified dimensions.

    Parameters:
    length: Number of sites
    phys_dim: Physical dimension at each site
    bond_dim: Maximum bond dimension
    dtype: Data type for tensors
    """
    tensors = []

    for i in range(length):
        if i == 0:  # First tensor
            left_dim = 1
            right_dim = min(bond_dim, phys_dim**(length-1))
        elif i == length - 1:  # Last tensor
            left_dim = min(bond_dim, phys_dim**(length-1))
            right_dim = 1
        else:  # Middle tensors
            left_dim = min(bond_dim, phys_dim**i)
            right_dim = min(bond_dim, phys_dim**(length-1-i))

        # Create random tensor
        if dtype in [np.complex64, np.complex128]:
            tensor = (np.random.randn(left_dim, phys_dim, right_dim) +
                     1j * np.random.randn(left_dim, phys_dim, right_dim)).astype(dtype)
        else:
            tensor = np.random.randn(left_dim, phys_dim, right_dim).astype(dtype)

        tensors.append(tensor)

    return MPS(tensors)

def create_product_state(length: int, state_indices: List[int]) -> MPS:
    """
    Create an MPS representing a product state.

    Parameters:
    length: Number of sites
    state_indices: List of local state indices (e.g., [0,1,0,1] for |0101⟩)
    """
    if len(state_indices) != length:
        raise ValueError("Length of state_indices must match MPS length")

    tensors = []
    phys_dim = max(state_indices) + 1

    for i, state_idx in enumerate(state_indices):
        if i == 0:  # First tensor
            tensor = np.zeros((1, phys_dim, 1))
            tensor[0, state_idx, 0] = 1.0
        elif i == length - 1:  # Last tensor
            tensor = np.zeros((1, phys_dim, 1))
            tensor[0, state_idx, 0] = 1.0
        else:  # Middle tensors
            tensor = np.zeros((1, phys_dim, 1))
            tensor[0, state_idx, 0] = 1.0

        tensors.append(tensor)

    return MPS(tensors)

# Create product state MPS
print("\n1. Creating product state |01010⟩...")
product_mps = create_product_state(5, [0, 1, 0, 1, 0])
print(f"   {product_mps}")
print(f"   Bond dimensions: {product_mps.bond_dims()}")
print(f"   Length: {len(product_mps)}")
print(f"   Max bond dimension: {product_mps.max_bond_dim()}")

# Create complex MPS
print("\n2. Creating complex MPS...")
complex_mps = create_random_mps(5, 2, 3, dtype=np.complex128)
print(f"   {complex_mps}")
print(f"   First tensor dtype: {complex_mps[0].dtype}")
print(f"   Second tensor with shape {complex_mps[2].shape} is \n{complex_mps[2]}")


1. Creating product state |01010⟩...
   MPS(sites=5, phys_dim=2, max_bond=1, dtype=float64)
   Bond dimensions: [1, 1, 1, 1, 1, 1]
   Length: 5
   Max bond dimension: 1

2. Creating complex MPS...
   MPS(sites=5, phys_dim=2, max_bond=3, dtype=complex128)
   First tensor dtype: complex128
   Second tensor with shape (3, 2, 3) is 
[[[ 0.19001169-0.69181477j -1.28610116-0.43278305j
    0.77406309-1.24505478j]
  [-0.60048458+0.80935027j  0.63268838-1.41878099j
    0.25725809+1.89930633j]]

 [[ 1.60369413+0.0191521j  -0.23943453+0.54623168j
   -0.48779423+0.1960289j ]
  [ 0.62968975-0.52510844j -0.87793804+0.50020047j
    0.57240086+1.24196251j]]

 [[ 0.47194141+0.00446272j -0.28680118-0.51450513j
    0.84590173-0.59271517j]
  [-1.43323792+0.24954991j -0.26706798+0.36312237j
    0.20621758-0.98260927j]]]


In [ ]:
mps_copy = product_mps.copy()

array([[[1.],
        [0.]]])

## Problem 1.

Write the MPS corresponding to the GHZ state: write a function that takes the length L as an input and outputs the GHZ state as an MPS.

# Demo 2: Convert MPS to dense

In [ ]:
import numpy as np
from typing import List

def mps_to_dense(M):
    """
    Convert an MPS to its dense state vector representation.
    WARNING: Only run this for small systems!! Memory scales as d^L.

    Parameters:
    M: MPS object

    Returns:
    numpy array: Dense state vector of length d^L
    """
    d = M.phys_dim()  # Physical dimension
    L = len(M)        # Number of sites

    # Start with the first tensor
    tmp = M.tensor_list[0]  # Shape: (1, d, bond_dim)

    # Contract with remaining tensors
    for i in range(1, L):
        A = M.tensor_list[i]  # Shape: (bond_dim, d, bond_dim)
        dR = A.shape[2]       # Right bond dimension

        # Einstein summation: tmp[l,s1,r1] * A[r1,s2,r] -> tmp2[l,s1,s2,r]
        # This contracts the right bond of tmp with left bond of A
        tmp2 = np.einsum('lsr,rtu->lstu', tmp, A)

        # Reshape to combine physical indices
        # tmp2 has shape (1, d, d, dR) -> reshape to (1, d^(i+1), dR)
        tmp = tmp2.reshape(1, d**(i+1), dR)

    # Final reshape to get the dense state vector
    psi = tmp.reshape(d**L)

    return psi

def mps_to_dense_verbose(M):
    """
    Verbose version with step-by-step explanation and intermediate shapes.
    """
    print("Converting MPS to dense state vector...")
    print(f"Physical dimension d = {M.phys_dim()}")
    print(f"Number of sites L = {len(M)}")
    print(f"Final vector size will be d^L = {M.phys_dim()**len(M)}")
    print()

    d = M.phys_dim()
    L = len(M)

    tmp = M.tensor_list[0]
    print(f"Step 0: Start with first tensor, shape = {tmp.shape}")

    for i in range(1, L):
        A = M.tensor_list[i]
        print(f"Step {i}: Contracting with tensor {i}, shape = {A.shape}")

        # Show the contraction explicitly
        print(f"  Before contraction: tmp.shape = {tmp.shape}")
        print(f"  Contracting indices: tmp[..., r1] * A[r1, ...]")

        tmp2 = np.einsum('lsr,rtu->lstu', tmp, A)
        print(f"  After contraction: tmp2.shape = {tmp2.shape}")

        # Reshape
        new_shape = (1, d**(i+1), A.shape[2])
        tmp = tmp2.reshape(new_shape)
        print(f"  After reshape: tmp.shape = {tmp.shape}")
        print()

    psi = tmp.reshape(d**L)
    print(f"Final dense vector shape: {psi.shape}")

    return psi

def verify_mps_conversion():
    """
    Create a simple MPS and verify the conversion makes sense.
    """
    print("=== Verification Example ===")

    # Create a simple 3-site MPS representing |000⟩ state
    # Each tensor puts all weight on the |0⟩ state
    tensor1 = np.zeros((1, 2, 2))  # (1, phys, bond)
    tensor1[0, 0, 0] = 1.0  # |0⟩ state, first bond index

    tensor2 = np.zeros((2, 2, 2))  # (bond, phys, bond)
    tensor2[0, 0, 0] = 1.0  # Continue |0⟩ state

    tensor3 = np.zeros((2, 2, 1))  # (bond, phys, 1)
    tensor3[0, 0, 0] = 1.0  # End with |0⟩ state

    mps = MPS([tensor1, tensor2, tensor3])

    print("Created 3-site MPS representing |000⟩ state")
    print("Tensor shapes:", [t.shape for t in mps.tensor_list])

    # Convert to dense
    psi = mps_to_dense(mps)
    print(f"Dense vector: {psi}")
    print(f"Expected: [1, 0, 0, 0, 0, 0, 0, 0] (|000⟩ in computational basis)")
    print(f"Norm: {np.linalg.norm(psi)}")

    return psi

if __name__ == "__main__":

    # Run verification
    psi = verify_mps_conversion()


=== Verification Example ===
Created 3-site MPS representing |000⟩ state
Tensor shapes: [(1, 2, 2), (2, 2, 2), (2, 2, 1)]
Dense vector: [1. 0. 0. 0. 0. 0. 0. 0.]
Expected: [1, 0, 0, 0, 0, 0, 0, 0] (|000⟩ in computational basis)
Norm: 1.0


# # Demo 3. Convert dense vector to MPS.

In [ ]:


def dense_to_MPS(psi: np.ndarray, d: int) -> MPS:
    """
    Convert a dense vector to Matrix Product State (MPS) representation

    Parameters:
    psi: Dense state vector
    d: Local physical dimension (physical dimension)

    Returns:
    MyMPS object containing the tensor decomposition
    """
    L = int(round(np.log(len(psi)) / np.log(d)))
    psi = psi.reshape((d, d**(L-1)))

    U, S, Vt = np.linalg.svd(psi, full_matrices=False)
    A1 = U.reshape((1, d, U.shape[1]))

    Ts = [A1]

    for i in range(2, L+1):
        SVt = np.diag(S) @ Vt
        dL = len(S)
        SVt = SVt.reshape((dL * d, d**(L-i)))

        U, S, Vt = np.linalg.svd(SVt, full_matrices=False)
        dR = U.shape[1]
        A = U.reshape((dL, d, dR))
        Ts.append(A)

    M = MPS(Ts)
    return M

In [ ]:
# Create a simple 3-site MPS representing |000⟩ state
# Each tensor puts all weight on the |0⟩ state
tensor1 = np.zeros((1, 2, 2))  # (1, phys, bond)
tensor1[0, 0, 0] = 1.0  # |0⟩ state, first bond index

tensor2 = np.zeros((2, 2, 2))  # (bond, phys, bond)
tensor2[0, 0, 0] = 1.0  # Continue |0⟩ state

tensor3 = np.zeros((2, 2, 1))  # (bond, phys, 1)
tensor3[0, 0, 0] = 1.0  # End with |0⟩ state

mps = MPS([tensor1, tensor2, tensor3])

print("Created 3-site MPS representing |000⟩ state")
print("Tensor shapes:", [t.shape for t in mps.tensor_list])

# Convert to dense
psi = mps_to_dense(mps)
print(psi)

Created 3-site MPS representing |000⟩ state
Tensor shapes: [(1, 2, 2), (2, 2, 2), (2, 2, 1)]
[1. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
mps = dense_to_MPS(psi, 2)
print(f"1st tensor: \n {mps[0]}")
print(f"2nd tensor: \n {mps[1]}")
print(f"3rd tensor: \n {mps[2]}")

psi = mps_to_dense(mps)
print(psi)

1st tensor: 
 [[[1. 0.]
  [0. 1.]]]
2nd tensor: 
 [[[1. 0.]
  [0. 1.]]

 [[0. 0.]
  [0. 0.]]]
3rd tensor: 
 [[[1.]
  [0.]]

 [[0.]
  [0.]]]
[1. 0. 0. 0. 0. 0. 0. 0.]


## Problem 2.

Write code to compute the entanglement entropy across each cut for an input MPS.

Hint: Use the mps_to_dense function to convert the state into a dense vector. Then perform successive SVD (see the dense_to_mps function) and use the singular values to compute entanglement.